In [2]:
import numpy as np

In [3]:
with open("input.txt") as f:
    data = [line.strip() for line in f.readlines()]

In [10]:
seeds = [int(seed) for seed in data[0].split(": ")[1].split()]
seeds

[1263068588,
 44436703,
 1116624626,
 2393304,
 2098781025,
 128251971,
 2946842531,
 102775703,
 2361566863,
 262106125,
 221434439,
 24088025,
 1368516778,
 69719147,
 3326254382,
 101094138,
 1576631370,
 357411492,
 3713929839,
 154258863]

In [50]:
il = iter(data[1:])
maps = dict()
mapnames = dict()
index = 0
for line in il:
    if not line.strip():
        continue
    if 'map' in line:
        mapname = line.strip().split()[0]
        mapnames[index] = mapname
        M = []
        while True:
            try:
                line = next(il)
            except StopIteration:
                break
            if not line.strip():
                break
            M.append(list(map(int, line.split())))
        Ma = np.array(M)
        maps[index] = Ma[np.argsort(Ma[:, 1])]
        index += 1
print(mapnames)

{0: 'seed-to-soil', 1: 'soil-to-fertilizer', 2: 'fertilizer-to-water', 3: 'water-to-light', 4: 'light-to-temperature', 5: 'temperature-to-humidity', 6: 'humidity-to-location'}


In [46]:
# maps work destinationid, sourceid, range

In [47]:
rangemaps = dict()

In [49]:
map1 = maps[0]

In [31]:
seeds[0]

1263068588

In [32]:
map1

array([[ 952690728,          0,  129136847],
       [ 491535670,  129136847,  193613329],
       [1585688167,  322750176,  100508616],
       [1749120098,  423258792,   10963816],
       [ 799335774,  434222608,    1707610],
       [1285511345,  435930218,   30230711],
       [1275917650,  466160929,    9593695],
       [ 731512552,  475754624,   67823222],
       [1686196783,  543577846,   17264874],
       [ 685148999,  560842720,   46363553],
       [  73663477,  607206273,   70526923],
       [ 220408859,  677733196,   21167055],
       [ 380867146,  698900251,  110668524],
       [1381626519,  809568775,   38912495],
       [ 801043384,  848481270,  151647344],
       [1081827575, 1000128614,  167139129],
       [ 144190400, 1167267743,    4402289],
       [1760083914, 1171670032,   71964744],
       [ 241575914, 1243634776,  139291232],
       [ 148592689, 1382926008,   71816170],
       [1703461657, 1454742178,   45658441],
       [1315742056, 1500400619,   65884463],
       [14

In [37]:
bin_location = np.searchsorted(map1[:,1], seeds[0], side='left')
bin_location

19

In [39]:
map1[bin_location-1], seeds[0]

(array([ 241575914, 1243634776,  139291232]), 1263068588)

In [43]:
def is_between(value, lower_bound, upper_bound):
    return lower_bound <= value < upper_bound
destid, sourceid, r = map1[bin_location-1]
print(sourceid, sourceid+r, seeds[0])
is_between(seeds[0], sourceid, sourceid+r)

1243634776 1382926008 1263068588


True

In [44]:
maps.keys()

dict_keys(['seed-to-soil', 'soil-to-fertilizer', 'fertilizer-to-water', 'water-to-light', 'light-to-temperature', 'temperature-to-humidity', 'humidity-to-location'])

In [96]:
def find_location(maps, seed, verbose=True):
    if verbose:print("---", seed)
    for i in range(len(maps)):
        if verbose: print(i, mapnames[i])
        bin_location = np.searchsorted(maps[i][:,1], seed, side='right')
        destid, sourceid, r = maps[i][bin_location-1]
        if is_between(seed, sourceid, sourceid+r):
            if verbose: print(f"seed {seed:d} is mapped between sourceid {sourceid} which ranges until {sourceid+r}")
            diff = seed - sourceid
            seed = destid + diff
        else:
            if verbose:
                print(f"seed {seed} is not within range {sourceid}-{sourceid+r} so it will map to its own value")
                print(bin_location)
                print(maps[i][bin_location-1:bin_location+3])
    return seed

In [97]:
find_location(maps, seeds[0])

--- 1263068588
0 seed-to-soil
seed 1263068588 is mapped between sourceid 1243634776 which ranges until 1382926008
1 soil-to-fertilizer
seed 261009726 is mapped between sourceid 214837072 which ranges until 422379995
2 fertilizer-to-water
seed 2358599195 is mapped between sourceid 2342244856 which ranges until 2446205321
3 water-to-light
seed 3531556561 is mapped between sourceid 2811361157 which ranges until 3736283841
4 light-to-temperature
seed 1610709606 is mapped between sourceid 1598820754 which ranges until 1616000911
5 temperature-to-humidity
seed 1411253525 is mapped between sourceid 1243181274 which ranges until 1591451985
6 humidity-to-location
seed 3376079053 is mapped between sourceid 3291443897 which ranges until 3512929162


4001340211

In [98]:
locations = [ find_location(maps, seed, verbose=False) for seed in seeds ]

In [99]:
min(locations)

199602917

## question 2

In [175]:
ranges = np.c_[seeds[0::2], np.sum([seeds[0::2], (np.array(seeds[1::2])-1)], axis=0)]
ranges

array([[1263068588, 1307505290],
       [1116624626, 1119017929],
       [2098781025, 2227032995],
       [2946842531, 3049618233],
       [2361566863, 2623672987],
       [ 221434439,  245522463],
       [1368516778, 1438235924],
       [3326254382, 3427348519],
       [1576631370, 1934042861],
       [3713929839, 3868188701]])

In [ ]:
def check_range(maps, startid, finalid):
    for i in range(len(maps)):
        
        split_range(maps[i], startid, finalid)
        
        # get bin_location of startid
        bin_location = np.searchsorted(maps[i][:,1], startid, side='right')
        destid, sourceid, r = maps[i][bin_location-1]
        
        if is_between(seed, sourceid, sourceid+r):
            if verbose: print(f"seed {seed:d} is mapped between sourceid {sourceid} which ranges until {sourceid+r}")
            diff = seed - sourceid
            seed = destid + diff
        else:
            if verbose:
                print(f"seed {seed} is not within range {sourceid}-{sourceid+r} so it will map to its own value")
                print(bin_location)
                print(maps[i][bin_location-1:bin_location+3])
    return seed

In [208]:
def split_range(xmap, startid, finalid):
    """ this function will look whether startid and finalid fall within the same subrange and otherwise return 
    multiple ranges that map to different destination ids"""
    bin_location = np.searchsorted(xmap[:,1], startid, side='right')
    print("bin_location", bin_location)
    destid, sourceid, r = xmap[bin_location-1]
    
    if is_between(startid, sourceid, sourceid+r):
        print(f"start value {startid} falls within sourceid {sourceid} and {sourceid+r}")
        diff = startid - sourceid
        seed = destid + diff
    else:
        try:
            destid2, sourceid2, r2 = xmap[bin_location]
        except IndexError:
            # in this case there is no higher bin, so map to itself
            return [[startid, finalid]]
            
        print(f"start value {startid} falls outside range. next range starts at {sourceid2}")
        # check if final id also falls outside range
        if finalid < sourceid2:
            print(f"whole seed range {startid}-{finalid} falls outside ranges {sourceid}-{sourceid+r} and {sourceid2}-{sourceid2+r2}")
            return [[startid, finalid]]
        else:
            # sourceid2 < finalid. new range is sourceid2-finalid
            return [[startid, sourceid2-1]] + split_range(xmap, sourceid2, finalid)
    
    if (sourceid + r) > finalid:
        
        print(f"the seed range {startid}-{finalid} falls totally within range {sourceid}-{sourceid+r}")
        return [[seed, seed + (finalid-startid)]]
    else:
        # now sourceid + r < finalid. so we have to check the new range sourceid + r + 1 - finalid
        print(f"the seed range {startid}-{finalid} is larger than range {sourceid}-{sourceid+r-1}")
        print(f"new range {sourceid+r}-{finalid}")
        return [[seed, destid+r-1]] + split_range(xmap, sourceid+r, finalid)

In [209]:
new_ranges = ranges.copy()
for i in range(len(maps)):
    new_new_ranges = []
    for xrange in new_ranges:
        new_range = split_range(maps[i], *xrange)
        new_new_ranges.extend(new_range)
    new_ranges = new_new_ranges.copy()

---
bin_location 19
start value 1263068588 falls within sourceid 1243634776 and 1382926008
the seed range 1263068588-1307505290 falls totally within range 1243634776-1382926008
---
bin_location 16
start value 1116624626 falls within sourceid 1000128614 and 1167267743
the seed range 1116624626-1119017929 falls totally within range 1000128614-1167267743
---
bin_location 27
start value 2098781025 falls within sourceid 2052611543 and 2132404068
the seed range 2098781025-2227032995 is larger than range 2052611543-2132404067
new range 2132404068-2227032995
---
bin_location 28
start value 2132404068 falls within sourceid 2132404068 and 2195535041
the seed range 2132404068-2227032995 is larger than range 2132404068-2195535040
new range 2195535041-2227032995
---
bin_location 29
start value 2195535041 falls within sourceid 2195535041 and 2215661108
the seed range 2195535041-2227032995 is larger than range 2195535041-2215661107
new range 2215661108-2227032995
---
bin_location 30
start value 22156

In [210]:
np.min(new_ranges)

2254686

In [ ]:
def is_between(value, lower_bound, upper_bound):
    return lower_bound <= value < upper_bound


def split_range(xmap, startid, finalid):
    """ this function will look whether startid and finalid fall within the same subrange and otherwise return 
    multiple ranges that map to different destination ids"""
    bin_location = np.searchsorted(xmap[:,1], startid, side='right')
    destid, sourceid, r = xmap[bin_location-1]
    
    if is_between(startid, sourceid, sourceid+r):
        diff = startid - sourceid
        seed = destid + diff
    else:
        try:
            destid2, sourceid2, r2 = xmap[bin_location]
        except IndexError:
            return [[startid, finalid]]
            
        if finalid < sourceid2:
            return [[startid, finalid]]
        else:
            return [[startid, sourceid2-1]] + split_range(xmap, sourceid2, finalid)
    
    if (sourceid + r) > finalid:
        return [[seed, seed + (finalid-startid)]]
    else:
        return [[seed, destid+r-1]] + split_range(xmap, sourceid+r, finalid)
    
new_ranges = ranges.copy()
for i in range(len(maps)):
    new = []
    for xrange in new_ranges:
        new_range = split_range(maps[i], *xrange)
        new.extend(new_range)
    new_ranges = new.copy()

print(min(new_ranges))